In [21]:
import pandas
import asyncio
import sys
import traceback
from playwright.async_api import async_playwright, Playwright, expect
from collections.abc import Callable

wh = None
configs = {
    'NavigationDefaultTimeoutSec': 5,
    'PageDefaultTimeoutSec': 5,
}

class ManualLoopInterrupt(Exception):
    pass

In [22]:
class WebHandler:
    def __init__(self):
        self.callee = None
        
    async def main_loop(self):
        async with async_playwright() as playwright:
            global browser_context
            chromium = playwright.chromium
            browser_context = await chromium.launch_persistent_context('', headless=False)
            browser_context.set_default_navigation_timeout(configs['NavigationDefaultTimeoutSec'] * 1000)
            browser_context.set_default_timeout(configs['PageDefaultTimeoutSec'] * 1000)

            while True:
                try:
                    if isinstance(self.callee, Callable):
                        await self.callee(browser_context.pages[0])
                except ManualLoopInterrupt:
                    break
                except:
                    print(''.join(traceback.format_exception(*sys.exc_info())))
                finally:
                    self.callee = None
                    await asyncio.sleep(1)
                    
            await browser_context.close()

In [23]:
global wh
wh = WebHandler()
asyncio.create_task(wh.main_loop())

<Task pending name='Task-19' coro=<WebHandler.main_loop() running at /tmp/nix-shell-28104-0/ipykernel_41293/1186367678.py:5>>

<class 'playwright.async_api._generated.Page'>


In [24]:
async def test(page):
    # raise ManualLoopInterrupt()
    print(type(page))

wh.callee = test

In [25]:
async def test(page):
    # raise ManualLoopInterrupt()
    await page.goto('https://www.google.com')

wh.callee = test